In [29]:
from datasets import load_dataset

dataset = load_dataset("wikipedia", "20220301.en")


Found cached dataset wikipedia (/Users/galen/.cache/huggingface/datasets/wikipedia/20220301.en/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


  0%|          | 0/1 [00:00<?, ?it/s]

In [30]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 6458670
    })
})


In [35]:
# DatasetDict({
#     train: Dataset({
#         features: ['id', 'url', 'title', 'text'],
#         num_rows: 6458670
#     })
# })

# tokenize the first 1000 rows to estimate total token count
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

token_counts = 0
for i in range(10000):
    token_counts += len(tokenizer(dataset["train"][i]["text"])["input_ids"])

print(token_counts)
print(token_counts / 10000)
print(token_counts / 10000 * 6458670)

Token indices sequence length is longer than the specified maximum sequence length for this model (8727 > 512). Running this sequence through the model will result in indexing errors


48759818
4875.9818
31492357372.205997


In [31]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

In [32]:
import json

In [33]:
QA_pairs = []
# load from QA_pairs.json if it exists
if os.path.exists('QA_pairs.json'):
    with open('QA_pairs.json', 'r') as f:
        QA_pairs = json.load(f)

for num_articles, article in enumerate(dataset['train']):
    print(num_articles)
    # skip articles that already have QA pairs
    if any([pair[0][0] == article['id'] for pair in QA_pairs]):
        continue

    print(article['title'])
    print(article['text'][:100])
    text_content = article['text']
    text_chunks = [text_content[i:i+2000] for i in range(0, len(text_content), 2000)]
    for idx, snippet in enumerate(text_chunks):
        try:
            response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {
                "role": "system",
                "content": "Create a simple reading comprehension question-answer pair for the provided wikipedia snippets. \n\nThe question should be specific enough that someone could answer by first searching for the relevant information, without prior knowledge of the corresponding passage. If there would be ambiguity about what the question is asking without looking at the passage, it's a bad question.\n"
                },
                {
                "role": "user",
                "content": f"[ARTICLE: {article['title']}]\n\n[SNIPPET]\n{snippet}\n[END SNIPPET]\nRespond in the following format:\nQ: [question]\nA: [answer]"
                }
            ],
            temperature=0,
            max_tokens=1579,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
            )
            print(response['choices'][0]['message']['content'])
            pair = [[article['id'], idx], response['choices'][0]['message']['content']]
            QA_pairs.append(pair)
            if len(QA_pairs) % 10 == 0:
                print(f"Saving {len(QA_pairs)} pairs")
                with open('QA_pairs.json', 'w') as f:
                    json.dump(QA_pairs, f)
        except Exception as e:
            print(e)
            continue


    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

KeyboardInterrupt: 